In [1]:
import math
import numpy as np
import pickle
import operator
pickle_in = open("model.pyc","rb")
model = pickle.load(pickle_in)
at = model['a']
b = model['b']
pi = model['pi']
class_set = list(at.keys())
file1, file2 = ['wsj19-21.testing','wsj19-21.truth']

In [34]:
word_dis_v = {}
for i in b:
    word_dis_v[i] = np.zeros(50)
    for j in b[i]:
        if j not in tag_c:
            tag_c.append(j)
        word_dis_v[i][tag_c.index(j)] = b[i][j]

In [39]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
save_object(word_dis_v, 'model_prior_distribution.pyc')


In [2]:
f = open(file1)
contents = f.read()
a = contents.split('\n')


# In[284]:


train_data = []
for i in a:
    new = list(i.split())
    train_data.append(new)


# In[280]:


s = train_data[0]
alpha = 1/20


In [3]:
def predict_v(s, alpha):
    d = {}
    path = {}
    for i in range(len(s)):
        d[i] = {}
        path[i] = {}
        for cla in class_set:
            d[i][cla] = -float('inf')
            path[i][cla] = []
        if i == 0:            
            for j in d[i]:
                if s[i] in b:
                    if j not in pi:
                        j = '00'
                    d[i][j] = math.log(pi[j]) + math.log(b[s[i]][j])+math.log(1-alpha)
                else:
                    if j not in pi:
                        j = '00'
                    d[i][j] = math.log(pi[j]) + math.log(alpha)
#                     else:
#                         d[i][j] = -10
                path[i][j] = [j]
        elif i>=1:
            for k in d[i]:
                tran = - float('inf')
                prev = []
                for m in d[i-1]:
                    if d[i-1][m] != -float('inf'):
                        #print(m,k)
                        if k not in at[m]:
                            k = '00'
                        new = d[i-1][m]+math.log(at[m][k])
                        
                        if new >= tran:
                            tran = new
                            prev = m
                if s[i] in b:
                    if k not in b[s[i]]:
                        k = '00'
                    d[i][k] = tran + math.log(b[s[i]][k])+math.log(1-alpha)
                else:
                    d[i][k] =tran + math.log(alpha)
                if prev != []:
                    path[i][k] = path[i-1][prev] +[k]
    a = []
    steps = 1
    while a == [] and len(s)-steps >= 0:
        a = path[len(s)-steps][max(d[len(s)-steps].items(), key=operator.itemgetter(1))[0]]
        steps +=1
    return a

In [7]:
s = train_data[0]
alpha = 1/20
s = train_data[0]
alpha = 1/20
def predict_v(s, alpha):
    d = {}
    path = {}
    for i in range(len(s)):
        d[i] = {}
        path[i] = {}
        for cla in class_set:
            d[i][cla] = -float('inf')
            path[i][cla] = []
        if i == 0:
            for j in d[i]:
                if s[i] in b:
                    if j in b[s[i]] and j in pi:
                        d[i][j] = math.log(pi[j]) + math.log(b[s[i]][j])+math.log(1-alpha)
                    else:
                        d[i][j] = -float('inf')
                else:
                    if j in pi:
                        d[i][j] = math.log(pi[j]) + math.log(alpha)
                    else:
                        d[i][j] = -10
                path[i][j] = [j]
        elif i>=1:
            for k in d[i]:
                tran = - float('inf')
                prev = []
                for m in d[i-1]:
                    if d[i-1][m] != -float('inf'):
                        #print(m,k)
                        if k in at[m]:
                            new = d[i-1][m]+math.log(at[m][k])
                        else:
                            new =  -float('inf')
                        if new > tran:
                            tran = new
                            prev = m
                if s[i] in b:
                    if k in b[s[i]]:
                        d[i][k] = tran + math.log(b[s[i]][k])+math.log(1-alpha)
                    else:
                        d[i][k] = -float('inf')
                else:
                    d[i][k] =tran + math.log(alpha)
                if prev != []:
                    path[i][k] = path[i-1][prev] +[k]
    a = []
    steps = 1
    while a == [] and len(s)-steps >= 0:
        a = path[len(s)-steps][max(d[len(s)-steps].items(), key=operator.itemgetter(1))[0]]
        steps +=1
    return a

In [14]:
example = ('I love nature language processing.').split()
predict_v(example, alpha)

['PRP', 'VBP', 'NN', 'NN', 'IN']

In [12]:
res = []
count = 0
not_able = []
len_eq = 0
for i in train_data:
    count += 1
    if not count%500:
        print('Finsh ' + str(count) + ' predictions')
    try:
        ans = predict_v(i, alpha)
        if len(ans) < len(i):
            new = predict_v(i[len(ans)+1:], alpha)
            ans += new
        if len(ans) < len(i):
            new = predict_v(i[len(ans)+1:], alpha)
            ans += new
        res.append(ans)
    except Exception:
        res.append([None])


# In[282]:


f = open(file2)
contents = f.read()
test_res = contents.split('\n')


# In[154]:


def file_separator(a):
    content = []
    tags = []
    for i in a:
        c1 = []
        t1 = []
        sentence = list(i.split())
        start = 0
        while start<len(sentence):
            c1.append(sentence[start])
            t1.append(sentence[start+1])
            start += 2
        content.append(c1)
        tags.append(t1)
    return [content,tags]
content,tags = file_separator(test_res)


# In[283]:


total = 0
correct = 0
for i in range(len(tags)):
    total += len(tags[i])
    for j in range(len(tags[i])):
        if j<len(res[i]):
            if res[i][j] == tags[i][j]:
                correct +=1
round(correct/total,3)
print("The accuracy is " + str(round(correct*100/total,1)) + '%.')

Finsh 500 predictions
Finsh 1000 predictions
Finsh 1500 predictions
Finsh 2000 predictions
Finsh 2500 predictions
Finsh 3000 predictions
Finsh 3500 predictions
Finsh 4000 predictions
Finsh 4500 predictions
Finsh 5000 predictions
Finsh 5500 predictions
The accuracy is 92.4%.


In [ ]:
total = 0
correct = 0
for i in range(len(tags)):
    total += len(tags[i])
    for j in range(len(tags[i])):
        if j<len(res[i]):
            if res[i][j] == tags[i][j]:
                correct +=1
round(correct/total,3)
print("The accuracy is " + str(round(correct*100/total,1)) + '%.')

In [ ]:
total = 0
correct = 0
for i in range(len(tags)):
    total += 1
    if res[i] == tags[i]:
        correct +=1
round(correct/total,3)
print("The accuracy is " + str(round(correct*100/total,1)) + '%.')